## Music Evaluation

In [5]:
import json
from argparse import ArgumentParser
import midi
import glob
import copy
import os
import numpy as np
import pretty_midi
from pprint import pprint
import pickle
from mgeval.mgeval import core, utils
from sklearn.model_selection import LeaveOneOut

In [6]:
# midi file이 들어있는 dict path를 넣어주세요!

# set1dir = '/home/bang/다운로드/maestro-v2.0.0-midi/maestro_midi'
# set2dir = '/home/bang/PycharmProjects/POP909-Dataset/pop909_midi'

set1dir = '/home/bang/PycharmProjects/MusicTransformer-Pytorch/MT_pop_1102_static/classic'
set2dir = '/home/bang/PycharmProjects/MusicTransformer-Pytorch/MT_pop_1102_static/pop'

In [7]:
set1 = glob.glob(os.path.join(set1dir, '*'))
set2 = glob.glob(os.path.join(set2dir, '*'))

## Number of samples

- set1: classic midi files
- set2: pop midi files

In [8]:
print("set1 size :", len(set1), ", set2 size :", len(set2))

set1 size : 909 , set2 size : 909


In [9]:
num_samples = min(len(set2), len(set1))

## metrics

- total used pitch
- pitch_range
- avg pitch shift
- avg IOI
- total_pitch class histogram

In [10]:
evalset = { 
            'total_used_pitch': np.zeros((num_samples, 1))
          , 'pitch_range': np.zeros((num_samples, 1))
          , 'avg_pitch_shift': np.zeros((num_samples, 1))
          , 'avg_IOI': np.zeros((num_samples, 1))
          #, 'total_used_note': np.zeros((num_samples, 1))
          # , 'bar_used_pitch': np.zeros((num_samples, args.num_bar, 1))
          # , 'bar_used_note': np.zeros((num_samples, args.num_bar, 1))
          , 'total_pitch_class_histogram': np.zeros((num_samples, 12))
          # , 'bar_pitch_class_histogram': np.zeros((num_samples, args.num_bar, 12))
          #, 'note_length_hist': np.zeros((num_samples, 12))
          #, 'pitch_class_transition_matrix': np.zeros((num_samples, 12, 12))
          #, 'note_length_transition_matrix': np.zeros((num_samples, 12, 12))
          }

In [11]:
metrics_list = list(evalset.keys())

single_arg_metrics = (
    [ 'total_used_pitch'
    , 'avg_IOI'
    , 'total_pitch_class_histogram'
    , 'pitch_range'
    ])

set1_eval = copy.deepcopy(evalset)
set2_eval = copy.deepcopy(evalset)

sets = [ (set1, set1_eval), (set2, set2_eval) ]

In [17]:
def pitch_range_All_instruments(feature):
    """
    pitch_range (Pitch range):
    The pitch range is calculated by subtraction of the highest and lowest used pitch in semitones.

    Returns:
    'p_range': a scalar for each sample.
    """
    
    max_pitch = 0
    min_pitch = 127
    
    instrument_num = len(feature['pretty_midi'].instruments)
    cnt_pitch_temp = []
    for i in range(instrument_num):
        piano_roll = feature['pretty_midi'].instruments[i].get_piano_roll(fs=100)
        cnt_pitch_temp.append(np.sum(piano_roll, axis=1))

    pitch_index = np.where(np.sum(cnt_pitch_temp, axis=0) > 0)

    p_range = np.max(pitch_index) - np.min(pitch_index)
    

    return p_range

## 평가 진행

In [18]:
# Extract Fetures
for _set, _set_eval in sets:
    for i in range(0, num_samples):
        feature = core.extract_feature(_set[i])
        for metric in metrics_list:
            if metric == 'pitch_range':
                evaluator = pitch_range_All_instruments2
            else:
                continue
                evaluator = getattr(core.metrics(), metric)
                
            if metric in single_arg_metrics:
                tmp = evaluator(feature)
            # elif metric in bar_metrics:
            #     # print(metric)
            #     tmp = evaluator(feature, 0, args.num_bar)
            # print(tmp.shape)
            else:
                tmp = evaluator(feature, 1)
            _set_eval[metric][i] = tmp

## 평가 결과

In [9]:
set1_TUP = (set1_eval['total_used_pitch']).sum(-1).mean()
set2_TUP = (set2_eval['total_used_pitch']).sum(-1).mean()

print("set1 total used pitch :", set1_TUP)
print("set2 total used pitch :", set2_TUP)

set1 total used pitch : 30.657865786578657
set2 total used pitch : 27.07150715071507


In [10]:
set1_TCU = (set1_eval['total_pitch_class_histogram']!=0).sum(-1).mean()
set2_TCU = (set2_eval['total_pitch_class_histogram']!=0).sum(-1).mean()

print("set1_TCU :", set1_TCU)
print("set2_TCU :", set2_TCU)

set1_TCU : 9.864686468646864
set2_TCU : 8.471947194719473


In [20]:
set1_PR = (set1_eval['pitch_range']).mean()
set2_PR = (set2_eval['pitch_range']).mean()

print("set1_PR :", set1_PR)
print("set2_PR :", set2_PR)

set1_PR : 53.34653465346535
set2_PR : 45.182618261826185


In [12]:
set1_APS = set1_eval['avg_pitch_shift'].sum(-1).mean()
set2_APS = set2_eval['avg_pitch_shift'].sum(-1).mean()

print("set1_APS :", set1_APS)
print("set2_APS :", set2_APS)

set1_APS : 8.886880998879453
set2_APS : 10.80890925040531


In [13]:
set1_IOI = set1_eval['avg_IOI'].sum(-1).mean()
set2_IOI = set2_eval['avg_IOI'].sum(-1).mean()

print("set1_IOI :", set1_IOI)
print("set2_IOI :", set2_IOI)

set1_IOI : 0.15001880846299548
set2_IOI : 0.17643044690673612


In [14]:
set1_eval['pitch_range']

array([[44.],
       [55.],
       [43.],
       [51.],
       [58.],
       [40.],
       [44.],
       [48.],
       [55.],
       [48.],
       [48.],
       [48.],
       [45.],
       [38.],
       [35.],
       [40.],
       [55.],
       [59.],
       [45.],
       [62.],
       [45.],
       [44.],
       [47.],
       [62.],
       [43.],
       [46.],
       [31.],
       [57.],
       [56.],
       [53.],
       [41.],
       [55.],
       [60.],
       [38.],
       [47.],
       [55.],
       [41.],
       [60.],
       [50.],
       [53.],
       [48.],
       [71.],
       [64.],
       [40.],
       [43.],
       [60.],
       [50.],
       [58.],
       [43.],
       [45.],
       [68.],
       [43.],
       [65.],
       [48.],
       [43.],
       [47.],
       [71.],
       [55.],
       [43.],
       [45.],
       [53.],
       [45.],
       [49.],
       [36.],
       [48.],
       [52.],
       [48.],
       [40.],
       [41.],
       [47.],
       [48.],
      